In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from geopy.geocoders import Nominatim
from math import radians, cos, sin, asin, sqrt
from numpy import cos, sin, arcsin, sqrt
from ast import literal_eval
from tqdm import tqdm
import requests
import json
import networkx as nx
import osmnx as ox
import requests
from geopy import Point
from geopy.distance import geodesic
import numpy as np

In [ ]:
df_building = pd.read_csv('ManilaBuildingsFinal.csv')

In [ ]:
df_building.rename(columns={'name' : 'Building'}, inplace = True)

In [ ]:
df_civic = pd.read_csv('CivicDistance.csv')
df_fire = pd.read_csv('FireDistance.csv')
df_government = pd.read_csv('GovernmentDistance.csv')
df_hospital = pd.read_csv('HospitalDistance.csv')
df_public = pd.read_csv('PublicDistance.csv')
df_school = pd.read_csv('SchoolDistance.csv')
df_sports = pd.read_csv('SportsDistance.csv')
df_supermarket = pd.read_csv('SupermarketDistance.csv')
df_toilet = pd.read_csv('ToiletDistance.csv')
df_transport = pd.read_csv('TransportDistance.csv')

In [ ]:
df_civic = df_civic.groupby('InputID').agg(lambda x: list(x))
df_fire = df_fire.groupby('InputID').agg(lambda x: list(x))
df_government = df_government.groupby('InputID').agg(lambda x: list(x))
df_hospital = df_hospital.groupby('InputID').agg(lambda x: list(x))
df_public = df_public.groupby('InputID').agg(lambda x: list(x))
df_school = df_school.groupby('InputID').agg(lambda x: list(x))
df_sports = df_sports.groupby('InputID').agg(lambda x: list(x))
df_supermarket = df_supermarket.groupby('InputID').agg(lambda x: list(x))
df_toilet = df_toilet.groupby('InputID').agg(lambda x: list(x))
df_transport = df_transport.groupby('InputID').agg(lambda x: list(x))

In [ ]:
df_civic = df_civic.reset_index()
df_fire = df_fire.reset_index()
df_government = df_government.reset_index()
df_hospital = df_hospital.reset_index()
df_public = df_public.reset_index()
df_school = df_school.reset_index()
df_sports = df_sports.reset_index()
df_supermarket = df_supermarket.reset_index()
df_toilet = df_toilet.reset_index()
df_transport = df_transport.reset_index()

In [ ]:
df_civic.drop(columns=['TargetID'], inplace = True)
df_fire.drop(columns=['TargetID'], inplace = True)
df_government.drop(columns=['TargetID'], inplace = True)
df_hospital.drop(columns=['TargetID'], inplace = True)
df_public.drop(columns=['TargetID'], inplace = True)
df_school.drop(columns=['TargetID'], inplace = True)
df_sports.drop(columns=['TargetID'], inplace = True)
df_supermarket.drop(columns=['TargetID'], inplace = True)
df_toilet.drop(columns=['TargetID'], inplace = True)
df_transport.drop(columns=['TargetID'], inplace = True)

In [ ]:
df_civic
df_fire
df_government
df_hospital
df_public
df_school
df_sports
df_supermarket
df_toilet
df_transport

In [ ]:
df_civic.rename(columns={'InputID':'Building', 'Distance':'Civic'}, inplace = True)
df_fire.rename(columns={'InputID':'Building', 'Distance':'Fire'}, inplace = True)
df_government.rename(columns={'InputID':'Building', 'Distance':'Government'}, inplace = True)
df_hospital.rename(columns={'InputID':'Building', 'Distance':'Hospital'}, inplace = True)
df_public.rename(columns={'InputID':'Building', 'Distance':'Public'}, inplace = True)
df_school.rename(columns={'InputID':'Building', 'Distance':'School'}, inplace = True)
df_sports.rename(columns={'InputID':'Building', 'Distance':'Sports'}, inplace = True)
df_supermarket.rename(columns={'InputID':'Building', 'Distance':'Supermarket'}, inplace = True)
df_toilet.rename(columns={'InputID':'Building', 'Distance':'Toilet'}, inplace = True)
df_transport.rename(columns={'InputID':'Building', 'Distance':'Transport'}, inplace = True)

In [ ]:
df = df_building.merge(df_civic, on='Building', how = 'inner')
df = df.merge(df_fire, on='Building', how = 'inner')
df = df.merge(df_government, on='Building', how = 'inner')
df = df.merge(df_hospital, on='Building', how = 'inner')
df = df.merge(df_public, on='Building', how = 'inner')
df = df.merge(df_school, on='Building', how = 'inner')
df = df.merge(df_sports, on='Building', how = 'inner')
df = df.merge(df_supermarket, on='Building', how = 'inner')
df = df.merge(df_toilet, on='Building', how = 'inner')
df = df.merge(df_transport, on='Building', how = 'inner')

In [ ]:
df

In [ ]:
df.columns

In [ ]:
df[['Civic', 'Fire', 'Government', 'Hospital', 'Public', 'School', 'Sports', 'Supermarket', 'Toilet', 'Transport']] = df[['Civic', 'Fire', 'Government', 'Hospital', 'Public', 'School', 'Sports', 'Supermarket', 'Toilet', 'Transport']].applymap(lambda x: np.divide(x, 10000) if isinstance(x, list) else x)

In [ ]:
#df[['Civic', 'Government']] = df[['Civic', 'Government']].applymap(lambda x: np.divide(x, 10000) if isinstance(x, list) else x)

In [ ]:
distance_map = {
    'Civic': 1.5,
    'Fire': 1.0,
    'Government': 10.0,
    'Hospital': 10.0,
    'Public': 2.0,
    'School': 1.0,
    'Sports': 1.5,
    'Supermarket': 1.0,
    'Toilet': 0.1,
    'Transport': 2.0,
}
for facility, max_dist in distance_map.items():
    try:
        df[f'{facility}_nearby'] = df[facility].apply(lambda lst: [d for d in lst if d <= max_dist])
    except:
        pass


for facility, max_dist in distance_map.items():
    try:
        df[f'{facility}_nearby'] = df[f'{facility}_nearby'].apply(lambda lst: [round(max_dist - d, 2) for d in lst])
    except:
        pass


for facility in distance_map:
    try:
        df[f'{facility}_Score'] = df[f'{facility}_nearby'].apply(sum)
    except:
        pass

from sklearn.preprocessing import MinMaxScaler

# Select score columns
score_cols = [col for col in df.columns if col.endswith('_Score')]

# Initialize scaler with feature range 0-10
scaler = MinMaxScaler(feature_range=(0, 10))

# Fit and transform, then round to 2 decimals
df[score_cols] = pd.DataFrame(
    scaler.fit_transform(df[score_cols]),
    columns=score_cols,
    index=df.index
).round(2)


In [ ]:
df.columns

In [ ]:
df.to_csv('ManilaScoreRAW1.csv', index = False)

In [ ]:
%%time
columns = ['Civic', 'Fire', 'Government', 'Hospital', 'Public', 
           'School', 'Sports', 'Supermarket', 'Toilet', 'Transport']

for col in columns:
    df[f'{col}_1km'] = None
    df[f'{col}_5km'] = None
    df[f'{col}_10km'] = None


In [ ]:
%%time
# Define columns
#columns = ['Civic', 'Fire', 'Government', 'Hospital', 'Public', 'School', 'Sports', 'Supermarket', 'Toilet', 'Transport']
columns = ['Transport']

# Define a function to filter numbers based on distance
def filter_distances(row, columns):
    for col in columns:
        row[f'{col}_1km'] = [number for number in row[col] if number <= 1]
        row[f'{col}_5km'] = [number for number in row[col] if number <= 5]
        row[f'{col}_10km'] = [number for number in row[col] if number <= 10]
    return row

# Apply the function across the dataframe
df = df.apply(lambda row: filter_distances(row, columns), axis=1)


In [ ]:
df

In [ ]:
%%time
# Define columns
#columns = ['Civic', 'Fire', 'Government', 'Hospital', 'Public', 'School', 'Sports', 'Supermarket', 'Toilet', 'Transport']
columns = ['Transport']
# Define a function to apply element-wise
def transform_list(lst, val):
    return [round(val - x, 2) for x in lst]

# Use applymap to vectorize calculations
for col in columns:
    df[f'{col}_1km'] = df[f'{col}_1km'].map(lambda x: transform_list(x, 1))
    df[f'{col}_5km'] = df[f'{col}_5km'].map(lambda x: transform_list(x, 5))
    df[f'{col}_10km'] = df[f'{col}_10km'].map(lambda x: transform_list(x, 10))



In [ ]:
%%time
# Define columns
#columns = ['Civic', 'Fire', 'Government', 'Hospital', 'Public', 'School', 'Sports', 'Supermarket', 'Toilet', 'Transport']
columns = ['Transport']

# Initialize score columns
for col in columns:
    df[f'{col}_1km_Score'] = None
    df[f'{col}_5km_Score'] = None
    df[f'{col}_10km_Score'] = None

# Define a function to calculate scores
def calculate_scores(row, columns):
    for col in columns:
        row[f'{col}_1km_Score'] = sum(row[f'{col}_1km'])
        row[f'{col}_5km_Score'] = sum(row[f'{col}_5km'])
        row[f'{col}_10km_Score'] = sum(row[f'{col}_10km'])
    return row

# Apply the function across the dataframe
df = df.apply(lambda row: calculate_scores(row, columns), axis=1)


In [ ]:
df

In [ ]:
df.columns[72]

In [ ]:
# Vectorized scaling operation
df.iloc[:, 69:72] = df.iloc[:, 69:72].apply(lambda col: (col - col.min()) / (col.max() - col.min()) * 10, axis=0)


In [ ]:
# Optimized approach
try:
    df.iloc[:, 69:72] = df.iloc[:, 69:72].astype(float).round(2)
except Exception as e:
    print('Error:', e)


In [ ]:
df

In [ ]:
df.columns

In [ ]:
df = df[['Building', 'lon', 'lat', 'Civic_1km_Score', 'Civic_5km_Score',
       'Civic_10km_Score', 'Fire_1km_Score', 'Fire_5km_Score',
       'Fire_10km_Score', 'Government_1km_Score', 'Government_5km_Score',
       'Government_10km_Score', 'Hospital_1km_Score', 'Hospital_5km_Score',
       'Hospital_10km_Score', 'Public_1km_Score', 'Public_5km_Score',
       'Public_10km_Score', 'School_1km_Score', 'School_5km_Score',
       'School_10km_Score', 'Sports_1km_Score', 'Sports_5km_Score',
       'Sports_10km_Score', 'Supermarket_1km_Score', 'Supermarket_5km_Score',
       'Supermarket_10km_Score', 'Toilet_1km_Score', 'Toilet_5km_Score',
       'Toilet_10km_Score', 'Transport_1km_Score', 'Transport_5km_Score',
       'Transport_10km_Score']]

In [ ]:
df.to_csv('ManilaScoreRAW.csv', index = False)

In [ ]:
df1 = pd.read_csv('ManilaScoreRAW.csv')

In [ ]:
df1['Transport_10km_Score'] = None

In [ ]:
df1['Transport_10km_Score'] = df['Transport_10km_Score']

In [ ]:
df1